In [ ]:
import cv2
import numpy as np

# Load OpenPose model
net = cv2.dnn.readNetFromCaffe('pose_deploy_linevec.prototxt', 'pose_iter_440000.caffemodel')

def detect_keypoints(image_path):
    image = cv2.imread(image_path)
    inWidth = 368
    inHeight = 368
    inpBlob = cv2.dnn.blobFromImage(image, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)
    net.setInput(inpBlob)
    output = net.forward()
    
    H, W = output.shape[2], output.shape[3]
    points = []
    for i in range(15):  # 15 key points for COCO model
        probMap = output[0, i, :, :]
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
        x = (image.shape[1] * point[0]) / W
        y = (image.shape[0] * point[1]) / H
        if prob > 0.1:
            points.append((int(x), int(y)))
        else:
            points.append(None)
    return points

# Example usage
keypoints = detect_keypoints('user_photo.jpg')
print(keypoints)


In [ ]:
from PIL import Image

def prepare_garment_image(image_path):
    garment_image = Image.open(image_path).convert('RGBA')
    # Additional processing if needed
    return garment_image

# Example usage
garment = prepare_garment_image('garment_image.png')
garment.show()


In [ ]:
def align_and_scale_garment(user_image_path, garment_image_path, keypoints):
    user_image = Image.open(user_image_path).convert('RGBA')
    garment_image = Image.open(garment_image_path).convert('RGBA')

    # Example: Align shoulders (keypoints[2] and keypoints[5])
    if keypoints[2] and keypoints[5]:
        shoulder_width = abs(keypoints[2][0] - keypoints[5][0])
        scale_factor = shoulder_width / garment_image.width
        new_size = (int(garment_image.width * scale_factor), int(garment_image.height * scale_factor))
        garment_image = garment_image.resize(new_size, Image.ANTIALIAS)

        # Position garment
        garment_x = keypoints[2][0] - garment_image.width // 2
        garment_y = keypoints[2][1] - garment_image.height // 2
        user_image.paste(garment_image, (garment_x, garment_y), garment_image)

    return user_image

# Example usage
aligned_image = align_and_scale_garment('user_photo.jpg', 'garment_image.png', keypoints)
aligned_image.show()


In [ ]:
def overlay_garment(user_image_path, garment_image_path, keypoints):
    user_image = Image.open(user_image_path).convert('RGBA')
    garment_image = prepare_garment_image(garment_image_path)
    result_image = align_and_scale_garment(user_image_path, garment_image_path, keypoints)
    return result_image

# Example usage
result = overlay_garment('user_photo.jpg', 'garment_image.png', keypoints)
result.show()


In [ ]:
from PIL import Image

def align_and_scale_garment(user_image_path, garment_image_path, keypoints):
    user_image = Image.open(user_image_path).convert('RGBA')
    garment_image = Image.open(garment_image_path).convert('RGBA')

    if keypoints[2] and keypoints[5]:
        shoulder_width = abs(keypoints[2][0] - keypoints[5][0])
        scale_factor = shoulder_width / garment_image.width
        new_size = (int(garment_image.width * scale_factor), int(garment_image.height * scale_factor))
        garment_image = garment_image.resize(new_size, resample=Image.ANTIALIAS)

        garment_x = keypoints[2][0] - garment_image.width // 2
        garment_y = keypoints[2][1] - garment_image.height // 2
        user_image.paste(garment_image, (garment_x, garment_y), garment_image)

    return user_image

# Example usage
# Ensure 'user_photo.jpg' and 'garment_image.png' are valid paths to your images
keypoints = [(100, 200), (150, 250)]  # Replace with actual keypoints
result = align_and_scale_garment('user.png', 'garment.png', keypoints)
result.show()
